In [ ]:
# %% 0. Imports & seed ------------------------------------------------------
import math, random, copy, collections, warnings
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
from sklearn.manifold import TSNE
import timm
warnings.filterwarnings("ignore")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# %% 1. Hyper-params you may tweak ------------------------------------------
DATA_ROOT = Path("/kaggle/input/mangodata/Root/Original")  # Updated path from image
BATCH_PRE = 4          # Extremely small batch size for ViT (memory intensive)
ACC_STEPS = 32         # Very high accumulation steps to maintain effective batch size of 128
PRE_EPOCHS = 100       # BYOL pre-training epochs
BATCH_SUP = 2          # Extremely small supervised batch size for ViT
EPOCHS_SUP = 100       # Supervised fine-tuning epochs
LABELED_FRAC = 0.60    # % of train set with labels (70% labeled, 30% unlabeled)
LINEAR_ONLY = False    # False = fine-tune whole encoder
TEMP = 0.5             # NT-Xent temperature

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Check if data path exists
try:
    assert DATA_ROOT.exists(), f"❌ DATA_ROOT not found: {DATA_ROOT}"
    print(f"✅ DATA_ROOT found: {DATA_ROOT}")
except AssertionError as e:
    print(e)
    # Fallback paths to try
    fallback_paths = [
        Path("/kaggle/input/mangodata/Root/Original"),
        Path("./data/Root/Original"),
        Path("./Root/Original")
    ]
    for path in fallback_paths:
        if path.exists():
            DATA_ROOT = path
            print(f"✅ Using fallback path: {DATA_ROOT}")
            break
    else:
        raise FileNotFoundError("No valid data path found. Please check your data directory.")

# %% 2. Dataset root --------------------------------------------------------
base_ds = datasets.ImageFolder(DATA_ROOT, transform=None)
print(f"{len(base_ds)} images, {len(base_ds.classes)} classes → {base_ds.classes}")

# %% 3. Improved ViT-optimized augmentations --------------------------------
# Better normalization values for natural images
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# ViT-optimized augmentations for BYOL (ViT is more robust to strong augmentations)
byol_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.2, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ColorJitter(brightness=0.8, contrast=0.8, saturation=0.8, hue=0.2),  # Stronger for ViT
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0))], p=0.5),
    transforms.RandomApply([transforms.RandomRotation(degrees=45)], p=0.4),  # Stronger rotation for ViT
    transforms.RandomApply([transforms.RandomPerspective(distortion_scale=0.3)], p=0.3),
    transforms.RandomApply([transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.4), ratio=(0.3, 3.3))  # Stronger erasing for ViT
])

# Supervised training augmentations
sup_tf = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.1),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

class BYOLDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset
        
    def __len__(self):
        return len(self.subset)
    
    def __getitem__(self, idx):
        img, _ = self.subset[idx]
        # Generate two different augmented views
        view1 = byol_tf(img)
        view2 = byol_tf(img)
        return view1, view2

# %% 4. Improved stratified splits -------------------------------------------
indices = np.arange(len(base_ds))
labels = np.array([label for _, label in base_ds.imgs])

# First split: train vs (val + test)
train_idx, rest_idx = train_test_split(
    indices, test_size=0.3, stratify=labels, random_state=SEED
)

# Second split: val vs test from the rest
val_idx, test_idx = train_test_split(
    rest_idx, test_size=0.5, stratify=labels[rest_idx], random_state=SEED
)

print(f"Train: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

# Create subsets
train_base = torch.utils.data.Subset(base_ds, train_idx)
val_base = torch.utils.data.Subset(base_ds, val_idx)
test_base = torch.utils.data.Subset(base_ds, test_idx)

# Split training data into labeled and unlabeled portions
perm = np.random.RandomState(SEED).permutation(len(train_base))
n_labeled = max(len(base_ds.classes), int(LABELED_FRAC * len(train_base)))  # At least 1 per class
labeled_idx = perm[:n_labeled]
unlabeled_idx = perm[n_labeled:]

print(f"Labeled: {len(labeled_idx)}, Unlabeled: {len(unlabeled_idx)}")

class TransformDataset(Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform
    
    def __len__(self):
        return len(self.subset)
    
    def __getitem__(self, idx):
        img, label = self.subset[idx]
        return self.transform(img), label

# Create final datasets
labeled_ds = TransformDataset(torch.utils.data.Subset(train_base, labeled_idx), sup_tf)
val_ds = TransformDataset(val_base, test_tf)
test_ds = TransformDataset(test_base, test_tf)
byol_ds = BYOLDataset(base_ds)  # Use whole dataset for self-supervised learning

# %% 5. DataLoaders ---------------------------------------------------------
pre_loader = DataLoader(byol_ds, batch_size=BATCH_PRE, shuffle=True, 
                       num_workers=0, pin_memory=False, drop_last=True)
lab_loader = DataLoader(labeled_ds, batch_size=BATCH_SUP, shuffle=True,
                       num_workers=0, pin_memory=False)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False,  # Very small for ViT
                       num_workers=0, pin_memory=False)
test_loader = DataLoader(test_ds, batch_size=4, shuffle=False,  # Very small for ViT
                        num_workers=0, pin_memory=False)

print(f"BYOL: {len(byol_ds)}, Labeled: {len(labeled_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}")

# %% 6. Vision Transformer BYOL model ---------------------------------------
def get_vit_encoder():
    """Create ViT encoder without pretrained weights - using smallest ViT variant"""
    try:
        # First try ViT-Tiny/16
        model = timm.create_model('vit_tiny_patch16_224', pretrained=False, num_classes=0)
        print(f"✅ Created ViT-Tiny encoder with {sum(p.numel() for p in model.parameters()):,} parameters")
        return model
    except Exception as e:
        print(f"❌ Failed to create ViT-Tiny: {e}")
        print("🔄 Falling back to MobileNetV3-Small...")
        # Fallback to a very lightweight CNN
        model = timm.create_model('mobilenetv3_small_100', pretrained=False, num_classes=0)
        print(f"✅ Created MobileNetV3-Small encoder with {sum(p.numel() for p in model.parameters()):,} parameters")
        return model

# Create a test encoder to determine the embedding dimension
test_encoder = get_vit_encoder()
with torch.no_grad():
    test_input = torch.randn(1, 3, 224, 224)
    test_output = test_encoder(test_input)
    ENC_DIM = test_output.shape[1]
    
print(f"📏 Detected encoder embedding dimension: {ENC_DIM}")

PROJ_DIM = min(128, ENC_DIM // 2)    # Adaptive projection dimension
PRED_DIM = min(128, ENC_DIM // 2)    # Adaptive prediction dimension
EMA_DECAY = 0.996                    # Slightly higher EMA decay for ViT

# Clean up test encoder
del test_encoder, test_input, test_output
if torch.cuda.is_available():
    torch.cuda.empty_cache()

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, use_bn=True, dropout=0.1):
        super().__init__()
        self.use_bn = use_bn
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        if use_bn:
            self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.gelu = nn.GELU()  # GELU activation for ViT
        self.dropout = nn.Dropout(dropout) if dropout > 0 else None
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.linear1(x)
        if self.use_bn:
            x = self.bn1(x)
        x = self.gelu(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = self.linear2(x)
        return x

class BYOL_ViT(nn.Module):
    def __init__(self):
        super().__init__()
        # Online network
        self.online_encoder = get_vit_encoder()
        self.online_projector = MLP(ENC_DIM, 512, PROJ_DIM, dropout=0.1)  # Much smaller hidden dim
        self.online_predictor = MLP(PROJ_DIM, 512, PRED_DIM, dropout=0.1)  # Much smaller hidden dim
        
        # Target network (no gradients)
        self.target_encoder = copy.deepcopy(self.online_encoder)
        self.target_projector = copy.deepcopy(self.online_projector)
        
        # Disable gradients for target networks
        for param in self.target_encoder.parameters():
            param.requires_grad = False
        for param in self.target_projector.parameters():
            param.requires_grad = False
    
    @torch.no_grad()
    def update_target_network(self):
        """Update target network with exponential moving average"""
        for online_params, target_params in zip(self.online_encoder.parameters(), 
                                               self.target_encoder.parameters()):
            target_params.data = EMA_DECAY * target_params.data + (1 - EMA_DECAY) * online_params.data
            
        for online_params, target_params in zip(self.online_projector.parameters(), 
                                               self.target_projector.parameters()):
            target_params.data = EMA_DECAY * target_params.data + (1 - EMA_DECAY) * online_params.data
    
    def forward(self, view1, view2):
        # Online network forward pass
        online_repr1 = self.online_encoder(view1)
        online_repr2 = self.online_encoder(view2)
        
        online_proj1 = self.online_projector(online_repr1)
        online_proj2 = self.online_projector(online_repr2)
        
        online_pred1 = self.online_predictor(online_proj1)
        online_pred2 = self.online_predictor(online_proj2)
        
        # Target network forward pass (no gradients)
        with torch.no_grad():
            target_repr1 = self.target_encoder(view1)
            target_repr2 = self.target_encoder(view2)
            
            target_proj1 = self.target_projector(target_repr1)
            target_proj2 = self.target_projector(target_repr2)
        
        return online_pred1, online_pred2, target_proj1.detach(), target_proj2.detach()

def byol_loss_fn(pred, target):
    """BYOL loss function with L2 normalization"""
    pred = F.normalize(pred, dim=-1, p=2)
    target = F.normalize(target, dim=-1, p=2)
    return 2 - 2 * (pred * target).sum(dim=-1)

# %% 7. BYOL pre-training with ViT ------------------------------------------
# Clear any cached memory before starting training
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Enable mixed precision training to reduce memory usage
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
use_amp = torch.cuda.is_available()

byol_model = BYOL_ViT().to(DEVICE)

# ViT-optimized optimizer with lower learning rate
optimizer_pre = torch.optim.AdamW([
    {'params': byol_model.online_encoder.parameters(), 'lr': 1e-4},
    {'params': byol_model.online_projector.parameters(), 'lr': 1e-3},
    {'params': byol_model.online_predictor.parameters(), 'lr': 1e-3}
], weight_decay=1e-4)  # Higher weight decay for ViT

# Learning rate scheduler with warmup (important for ViT)
def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * progress)))
    
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

warmup_steps = PRE_EPOCHS // 5  # Longer warmup for ViT
scheduler_pre = get_cosine_schedule_with_warmup(optimizer_pre, warmup_steps, PRE_EPOCHS)

print(f"\n▶ BYOL pre-training with ViT ({PRE_EPOCHS} epochs)")
byol_losses = []

for epoch in range(1, PRE_EPOCHS + 1):
    byol_model.train()
    epoch_losses = []
    
    progress_bar = tqdm(pre_loader, desc=f"Pre-train Epoch {epoch:03d}/{PRE_EPOCHS}", leave=False)
    
    for step, (view1, view2) in enumerate(progress_bar):
        view1, view2 = view1.to(DEVICE, non_blocking=True), view2.to(DEVICE, non_blocking=True)
        
        try:
            # Forward pass with mixed precision
            if use_amp:
                with torch.cuda.amp.autocast():
                    pred1, pred2, target1, target2 = byol_model(view1, view2)
                    
                    # Compute BYOL loss (symmetric)
                    loss1 = byol_loss_fn(pred1, target2).mean()
                    loss2 = byol_loss_fn(pred2, target1).mean()
                    loss = loss1 + loss2
                    loss = loss / ACC_STEPS  # Gradient accumulation
                
                # Backward pass with mixed precision
                scaler.scale(loss).backward()
            else:
                # Forward pass
                pred1, pred2, target1, target2 = byol_model(view1, view2)
                
                # Compute BYOL loss (symmetric)
                loss1 = byol_loss_fn(pred1, target2).mean()
                loss2 = byol_loss_fn(pred2, target1).mean()
                loss = loss1 + loss2
                loss = loss / ACC_STEPS  # Gradient accumulation
                
                # Backward pass
                loss.backward()
            
            if (step + 1) % ACC_STEPS == 0:
                if use_amp:
                    # Gradient clipping with mixed precision
                    scaler.unscale_(optimizer_pre)
                    torch.nn.utils.clip_grad_norm_(byol_model.online_encoder.parameters(), 1.0)
                    torch.nn.utils.clip_grad_norm_(byol_model.online_projector.parameters(), 1.0)
                    torch.nn.utils.clip_grad_norm_(byol_model.online_predictor.parameters(), 1.0)
                    
                    scaler.step(optimizer_pre)
                    scaler.update()
                else:
                    # Gradient clipping for stability (important for ViT)
                    torch.nn.utils.clip_grad_norm_(byol_model.online_encoder.parameters(), 1.0)
                    torch.nn.utils.clip_grad_norm_(byol_model.online_projector.parameters(), 1.0)
                    torch.nn.utils.clip_grad_norm_(byol_model.online_predictor.parameters(), 1.0)
                    
                    optimizer_pre.step()
                
                optimizer_pre.zero_grad()
                byol_model.update_target_network()
            
            epoch_losses.append(loss.item() * ACC_STEPS)
            progress_bar.set_postfix({'loss': f'{loss.item() * ACC_STEPS:.4f}'})
            
            # Clear intermediate tensors to save memory
            del pred1, pred2, target1, target2, loss1, loss2, loss
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"Warning: OOM at step {step}, clearing cache and skipping batch")
                torch.cuda.empty_cache()
                optimizer_pre.zero_grad()
                continue
            else:
                raise e
        
        # Periodic memory cleanup
        if step % 50 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()
    # Step scheduler
    scheduler_pre.step()
    
    avg_loss = np.mean(epoch_losses)
    byol_losses.append(avg_loss)
    
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: BYOL loss = {avg_loss:.4f}, LR = {scheduler_pre.get_last_lr()[0]:.6f}")

# Save encoder weights
encoder_checkpoint = copy.deepcopy(byol_model.online_encoder).cpu().state_dict()

# Plot pre-training loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, PRE_EPOCHS + 1), byol_losses, 'b-', linewidth=2)
plt.title('BYOL Pre-training Loss (ViT Backbone)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

# %% 7.5. t-SNE Visualization and Silhouette Score Analysis ----------------
print("\n▶ Performing t-SNE visualization and silhouette analysis...")

def extract_features_for_analysis(model, data_loader, max_samples=100):  # Much smaller sample size
    """Extract features from the encoder for analysis"""
    model.eval()
    features = []
    labels_list = []
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(data_loader):
            if len(features) >= max_samples:
                break
            
            # Process one image at a time to avoid OOM
            for j in range(images.size(0)):
                if len(features) >= max_samples:
                    break
                
                single_img = images[j:j+1].to(DEVICE, non_blocking=True)
                try:
                    feat = model(single_img)
                    features.append(feat.cpu().numpy())
                    labels_list.append(labels[j].item())
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                except RuntimeError as e:
                    if "out of memory" in str(e):
                        print(f"OOM during feature extraction, skipping sample")
                        torch.cuda.empty_cache()
                        continue
                    else:
                        raise e
    
    if features:
        return np.vstack(features)[:max_samples], np.array(labels_list)[:max_samples]
    else:
        return np.array([]).reshape(0, ENC_DIM), np.array([])

# Extract features using the pre-trained ViT encoder
encoder_for_analysis = get_vit_encoder()
encoder_for_analysis.load_state_dict(encoder_checkpoint)
encoder_for_analysis.to(DEVICE)

# Create a sample of test data for analysis
analysis_features, analysis_labels = extract_features_for_analysis(encoder_for_analysis, test_loader, max_samples=100)

# Compute silhouette score
print("Computing silhouette score...")
if len(np.unique(analysis_labels)) > 1:
    silhouette_avg = silhouette_score(analysis_features, analysis_labels)
    print(f"🎯 Silhouette Score (ViT Pre-training): {silhouette_avg:.4f}")
    
    # Per-class silhouette scores
    from sklearn.metrics import silhouette_samples
    sample_silhouette_values = silhouette_samples(analysis_features, analysis_labels)
    
    print("\nPer-class silhouette scores:")
    for i in range(len(base_ds.classes)):
        class_silhouette = sample_silhouette_values[analysis_labels == i]
        if len(class_silhouette) > 0:
            print(f"  {base_ds.classes[i]}: {np.mean(class_silhouette):.4f}")

# Perform t-SNE visualization
print("Computing t-SNE embedding...")
tsne = TSNE(n_components=2, random_state=SEED, perplexity=30, n_iter=1000)
tsne_results = tsne.fit_transform(analysis_features)

# Plot t-SNE results
plt.figure(figsize=(12, 8))
colors = plt.cm.Set3(np.linspace(0, 1, len(base_ds.classes)))

for i, class_name in enumerate(base_ds.classes):
    mask = analysis_labels == i
    if np.any(mask):
        plt.scatter(tsne_results[mask, 0], tsne_results[mask, 1], 
                   c=[colors[i]], label=class_name, alpha=0.7, s=50)

plt.title('t-SNE Visualization of ViT Pre-trained Features')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# %% 8. Enhanced Supervised fine-tuning with ViT ---------------------------
# Initialize ViT encoder and classifier
encoder = get_vit_encoder().to(DEVICE)
encoder.load_state_dict(encoder_checkpoint)

if LINEAR_ONLY:
    for param in encoder.parameters():
        param.requires_grad = False
    print("Fine-tuning: Linear probe only (ViT)")
else:
    print("Fine-tuning: Full ViT model")

# Enhanced classifier with dropout
classifier = nn.Sequential(
    nn.Dropout(0.3),  # Lower dropout for ViT
    nn.Linear(ENC_DIM, len(base_ds.classes))
).to(DEVICE)

# Initialize classifier with Xavier uniform
nn.init.xavier_uniform_(classifier[1].weight)
nn.init.constant_(classifier[1].bias, 0)

# ViT-optimized optimizer with different learning rates
if LINEAR_ONLY:
    optimizer_sup = torch.optim.AdamW(classifier.parameters(), lr=1e-3, weight_decay=1e-4)
else:
    optimizer_sup = torch.optim.AdamW([
        {'params': encoder.parameters(), 'lr': 1e-5},  # Very low LR for ViT encoder
        {'params': classifier.parameters(), 'lr': 1e-3}  # Higher LR for classifier
    ], weight_decay=1e-4)

# Enhanced scheduler
scheduler_sup = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_sup, mode='max', factor=0.5, patience=15, verbose=True
)

# Loss function with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Training history
train_losses, val_losses, val_accuracies = [], [], []
best_val_acc = 0.0

print(f"\n▶ Enhanced Fine-tuning with ViT ({EPOCHS_SUP} epochs)")

for epoch in range(1, EPOCHS_SUP + 1):
    # Training phase
    encoder.train()
    classifier.train()
    train_loss_batch = []
    
    progress_bar = tqdm(lab_loader, desc=f"Fine-tune Epoch {epoch:03d}/{EPOCHS_SUP}", leave=False)
    
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
        
        try:
            # Forward pass
            features = encoder(images)
            logits = classifier(features)
            loss = criterion(logits, labels)
            
            # Backward pass
            optimizer_sup.zero_grad()
            loss.backward()
            
            # Gradient clipping (important for ViT)
            torch.nn.utils.clip_grad_norm_(encoder.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
            
            optimizer_sup.step()
            
            train_loss_batch.append(loss.item())
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # Clear intermediate tensors
            del features, logits, loss
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"Warning: OOM during training, clearing cache and skipping batch")
                torch.cuda.empty_cache()
                optimizer_sup.zero_grad()
                continue
            else:
                raise e
    
    # Validation phase
    encoder.eval()
    classifier.eval()
    val_loss_batch = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
            
            try:
                features = encoder(images)
                logits = classifier(features)
                loss = criterion(logits, labels)
                
                val_loss_batch.append(loss.item())
                
                _, predicted = torch.max(logits.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Clear intermediate tensors
                del features, logits, loss
                
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"Warning: OOM during validation, clearing cache and skipping batch")
                    torch.cuda.empty_cache()
                    continue
                else:
                    raise e
    
    # Periodic memory cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Calculate metrics
    avg_train_loss = np.mean(train_loss_batch)
    avg_val_loss = np.mean(val_loss_batch)
    val_acc = 100.0 * correct / total
    
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)
    
    # Step scheduler
    scheduler_sup.step(val_acc)
    
    # Track best validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # Save best model
        torch.save({
            'encoder': encoder.state_dict(),
            'classifier': classifier.state_dict(),
            'val_acc': val_acc
        }, 'best_vit_model.pth')
    
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss = {avg_train_loss:.4f}, "
              f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_acc:.2f}%, "
              f"Best Val Acc = {best_val_acc:.2f}%")

# Load best model
try:
    checkpoint = torch.load('best_vit_model.pth')
    encoder.load_state_dict(checkpoint['encoder'])
    classifier.load_state_dict(checkpoint['classifier'])
    print(f"Loaded best ViT model with validation accuracy: {checkpoint['val_acc']:.2f}%")
except:
    print("Using final ViT model (best model checkpoint not found)")

# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(range(1, len(train_losses) + 1), train_losses, 'b-', label='Train Loss', linewidth=2)
ax1.plot(range(1, len(val_losses) + 1), val_losses, 'r-', label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss (ViT)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(range(1, len(val_accuracies) + 1), val_accuracies, 'g-', label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Validation Accuracy (ViT)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% 8.5. Post-training t-SNE and Silhouette Analysis ----------------------
print("\n▶ Post-training ViT feature analysis...")

# Extract features from fine-tuned ViT encoder
post_features, post_labels = extract_features_for_analysis(encoder, test_loader, max_samples=100)

# Compute post-training silhouette score
if len(np.unique(post_labels)) > 1:
    post_silhouette_avg = silhouette_score(post_features, post_labels)
    print(f"🎯 Silhouette Score (ViT Post-training): {post_silhouette_avg:.4f}")
    print(f"📈 Silhouette Improvement: {post_silhouette_avg - silhouette_avg:.4f}")

# Post-training t-SNE
print("Computing post-training t-SNE embedding...")
post_tsne = TSNE(n_components=2, random_state=SEED, perplexity=30, n_iter=1000)
post_tsne_results = post_tsne.fit_transform(post_features)

# Plot comparison of pre-training vs post-training t-SNE
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Pre-training t-SNE
for i, class_name in enumerate(base_ds.classes):
    mask = analysis_labels == i
    if np.any(mask):
        ax1.scatter(tsne_results[mask, 0], tsne_results[mask, 1], 
                   c=[colors[i]], label=class_name, alpha=0.7, s=50)

ax1.set_title('t-SNE: ViT Pre-training Features')
ax1.set_xlabel('t-SNE Component 1')
ax1.set_ylabel('t-SNE Component 2')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Post-training t-SNE
for i, class_name in enumerate(base_ds.classes):
    mask = post_labels == i
    if np.any(mask):
        ax2.scatter(post_tsne_results[mask, 0], post_tsne_results[mask, 1], 
                   c=[colors[i]], label=class_name, alpha=0.7, s=50)

ax2.set_title('t-SNE: ViT Post-training Features')
ax2.set_xlabel('t-SNE Component 1')
ax2.set_ylabel('t-SNE Component 2')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% 9. Final evaluation ----------------------------------------------------
encoder.eval()
classifier.eval()

y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)
        
        try:
            features = encoder(images)
            logits = classifier(features)
            probs = F.softmax(logits, dim=1)
            
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(torch.argmax(logits, dim=1).cpu().tolist())
            y_probs.extend(probs.cpu().numpy())
            
            # Clear intermediate tensors
            del features, logits, probs
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"Warning: OOM during evaluation, clearing cache and skipping batch")
                torch.cuda.empty_cache()
                continue
            else:
                raise e

# Calculate test accuracy
test_accuracy = 100.0 * sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
print(f"\n🎯 Final Test Accuracy: {test_accuracy:.2f}%")

# Classification report
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=base_ds.classes, digits=3))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=base_ds.classes, yticklabels=base_ds.classes)
plt.title(f'Confusion Matrix - ViT (Test Accuracy: {test_accuracy:.2f}%)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# %% 10. Summary of Results -------------------------------------------------
print("\n" + "="*70)
print("📋 FINAL RESULTS SUMMARY (Memory-Optimized Backbone)")
print("="*70)
print(f"🎯 Final Test Accuracy: {test_accuracy:.2f}%")
print(f"🎯 Best Validation Accuracy: {best_val_acc:.2f}%")
if 'silhouette_avg' in locals() and len(analysis_features) > 0:
    print(f"📊 Pre-training Silhouette Score: {silhouette_avg:.4f}")
if 'post_silhouette_avg' in locals() and len(post_features) > 0:
    print(f"📊 Post-training Silhouette Score: {post_silhouette_avg:.4f}")
    print(f"📈 Silhouette Improvement: {post_silhouette_avg - silhouette_avg:.4f}")

print(f"\n📈 Training Configuration:")
print(f"   - Backbone: Auto-detected ({ENC_DIM}-dim embeddings)")
print(f"   - Pre-training epochs: {PRE_EPOCHS}")
print(f"   - Fine-tuning epochs: {EPOCHS_SUP}")
print(f"   - Labeled fraction: {LABELED_FRAC:.1%}")
print(f"   - Linear probe only: {LINEAR_ONLY}")
print(f"   - EMA decay: {EMA_DECAY}")
print(f"   - Batch sizes: Pre-train={BATCH_PRE}, Fine-tune={BATCH_SUP}")
print(f"   - Gradient accumulation steps: {ACC_STEPS}")
print(f"   - Mixed precision training: {use_amp}")

print(f"\n✅ Memory-optimized training completed successfully!")
print("="*70)

Using device: cuda
✅ DATA_ROOT found: /kaggle/input/mangodata/Root/Original
837 images, 6 classes → ['Amrapali Mango', 'Banana Mango', 'Chaunsa Mango', 'Fazli Mango', 'Haribhanga Mango', 'Himsagar Mango']
Train: 585, Val: 126, Test: 126
Labeled: 351, Unlabeled: 234
BYOL: 837, Labeled: 351, Val: 126, Test: 126
✅ Created ViT-Tiny encoder with 5,524,416 parameters
📏 Detected encoder embedding dimension: 192
✅ Created ViT-Tiny encoder with 5,524,416 parameters

▶ BYOL pre-training with ViT (100 epochs)


Pre-train Epoch 001/100:   0%|          | 0/209 [00:00<?, ?it/s]

Epoch 001: BYOL loss = 4.0276, LR = 0.000005


Pre-train Epoch 002/100:   0%|          | 0/209 [00:00<?, ?it/s]

Pre-train Epoch 003/100:   0%|          | 0/209 [00:00<?, ?it/s]

...

Pre-train Epoch 071/100:   0%|          | 0/209 [00:00<?, ?it/s]

Pre-train Epoch 072/100:   0%|          | 0/209 [00:00<?, ?it/s]

Pre-train Epoch 073/100:   0%|          | 0/209 [00:00<?, ?it/s]

Pre-train Epoch 074/100:   0%|          | 0/209 [00:00<?, ?it/s]

Pre-train Epoch 075/100:   0%|          | 0/209 [00:00<?, ?it/s]